In [1]:
#Selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


#Time
from time import sleep

import pandas as pd
import itertools

In [2]:
def setupLinkedIn(email, password,sleep_time):
    #Setup   
    selEmail = driver.find_element(By.ID, 'session_key')
    selPW = driver.find_element(By.ID, 'session_password')
    selEmail.send_keys(email)
    selPW.send_keys(password)
    selPW.send_keys(Keys.RETURN)

    sleep(sleep_time)


In [3]:
def goToPage(page_number, sleep_time):
    #Go to the initial page
    baseUrl = 'https://www.linkedin.com/search/results/companies/?companyHqGeo=%5B%22101174742%22%5D&origin=GLOBAL_SEARCH_HEADER'
    pageUrl = baseUrl + f'&page={page_number}'
    driver.get(pageUrl)
    
    sleep(sleep_time)
    

In [4]:
def getCompanyPageLinks(sleep_time):
    """
    Requirements: be on a company search page
    Should be filtered by country Canada first, but will work on any company search page
    """
    companyWebElements = driver.find_elements(By.XPATH, "//span[@class='entity-result__title-text t-16']//a[@href]")
    companyLinks = []
    for elems in companyWebElements:
        companyLinks.append(elems.get_attribute('href'))
        
    sleep(sleep_time)
    return companyLinks

In [5]:
def loopThroughPageLinks(page_number_list: list, sleep_time):
    
    ###
    # First loop: get all of the company links
    ###
    all_links = []
    #Loop through every page
    for page_number in page_number_list:
        
        goToPage(page_number = page_number, sleep_time = sleep_time)
        
        all_links.append(getCompanyPageLinks(sleep_time))
    
    return all_links

In [46]:
def getAboutText(company_link, sleep_time):
    """
    Gets the about text and company name
    """
    #Go to company about page
    about_link = company_link + 'about/'
    driver.get(about_link)

    sleep(sleep_time)
    aboutText = driver.find_element(By.XPATH, "//div[@class='ember-view']//section[@class='artdeco-card p5 mb4']")
    companyName = driver.find_element(By.XPATH, "//*[contains(@class, 'ember-view t-24 t-black t-bold')]")
    return aboutText.text, companyName.text

In [7]:
def loopThroughAboutTexts(company_links_nested, sleep_time):
    output_list = []
    for i, link in enumerate(list(itertools.chain.from_iterable(company_links_nested))):
        abouttext, companyname = getAboutText(company_link = link, sleep_time = sleep_time)
        output_list.append({'id' : i,
                         'Name' : companyname,
                         'Link' : link,
                         'aboutText' : abouttext})
    return output_list

In [48]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://linkedin.com')
sleep(3)



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\louis\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
C:\Users\louis\AppData\Local\Temp\ipykernel_4916\1904361333.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [49]:
#Sign in to Feed Page
setupLinkedIn(email = 'louis.dhulst@mail.mcgill.ca', password = 'zvkDry%ZdCX5Q7e',sleep_time = 5)

links = loopThroughPageLinks(page_number_list = list(range(1,3)), sleep_time = 5)

linkedIn_text = loopThroughAboutTexts(company_links_nested = links, sleep_time = 5)

In [51]:
desired_fields = ['Overview', 'Website','Industry','Company size','Headquarters','Specialties']
cleanedText = []
pointer = None


for company_dict in linkedIn_text:
    #Initialize empty dict with fields
    cleanedText.append({'Name':company_dict['Name'],
                       'LinkedIn Link':company_dict['Link']})
    cleanedText[-1].update({key: None for key in desired_fields})
    
    for text in company_dict['aboutText'].split("\n"):

        #Continue for empty string
        if not text:
            continue

        #Update pointer
        if text in desired_fields:
            pointer = text
            continue
        #Put in text  
        if pointer:
            if not cleanedText[-1][pointer]:
                cleanedText[-1][pointer] = text
            else:
                cleanedText[-1][pointer] += ' ' + text
        

In [52]:
scrapedAbouts = pd.DataFrame(cleanedText)

In [53]:
scrapedAbouts

,Name,LinkedIn Link,Overview,Website,Industry,Company size,Headquarters,Specialties
0,Amazon,https://www.linkedin.com/company/amazon/,Amazon is guided by four principles: customer ...,https://www.aboutamazon.com/,"Technology, Information and Internet","10,001+ employees 1,008,728 on LinkedIn Includ...","Seattle, WA","e-Commerce, Retail, Operations, and Internet"
1,Microsoft,https://www.linkedin.com/company/microsoft/,Every company has a mission. What's ours? To e...,https://news.microsoft.com/,Software Development,"10,001+ employees 220,880 on LinkedIn Includes...","Redmond, Washington","Business Software, Developer Tools, Home & Edu..."
2,JPMorgan Chase & Co.,https://www.linkedin.com/company/jpmorganchase/,"For over 200 years, JPMorgan Chase & Co has pr...",http://www.jpmorganchase.com,Financial Services,"10,001+ employees 313,727 on LinkedIn Includes...","New York, NY","Consumer & Community Banking, Corporate and In..."
3,PepsiCo,https://www.linkedin.com/company/pepsico/,PepsiCo products are enjoyed by consumers more...,http://www.pepsico.com,Food and Beverage Services,"10,001+ employees 167,749 on LinkedIn Includes...","Purchase, New York","manufacturing, sales, marketing, brand managem..."
4,Google,https://www.linkedin.com/company/google/,A problem isn't truly solved until it's solved...,https://goo.gle/3m1IN7m,"Technology, Information and Internet","10,001+ employees 292,819 on LinkedIn Includes...","Mountain View, CA","search, ads, mobile, android, online video, ap..."
5,Cisco,https://www.linkedin.com/company/cisco/,Cisco (NASDAQ: CSCO) enables people to make po...,http://www.cisco.com,Software Development,"10,001+ employees 99,051 on LinkedIn Includes ...","San Jose, CA","Networking, Wireless, Security, Unified Commun..."
6,LinkedIn,https://www.linkedin.com/company/linkedin/,"Founded in 2003, LinkedIn connects the world's...",https://careers.linkedin.com,"Technology, Information and Internet","10,001+ employees 25,179 on LinkedIn Includes ...","Sunnyvale, CA","Online Professional Network, Jobs, People Sear..."
7,Pfizer,https://www.linkedin.com/company/pfizer/,Breakthroughs That Change Patients' Lives: Pfi...,http://www.pfizer.com,Pharmaceutical Manufacturing,"10,001+ employees 124,192 on LinkedIn Includes...","New York, New York",To apply science and our global resources to i...
8,Johnson & Johnson,https://www.linkedin.com/company/johnson-&-joh...,Profound Change Requires Boldness. Johnson & ...,http://www.jnj.com,Hospitals and Health Care,"10,001+ employees 140,485 on LinkedIn Includes...","New Brunswick, NJ","Health Care, Medical Devices & Diagnostics, Ph..."
9,Citi,https://www.linkedin.com/company/citi/,Citi's mission is to serve as a trusted partne...,http://www.citigroup.com,Financial Services,"10,001+ employees 237,860 on LinkedIn Includes...","New York, New York Founded 1812","Banking, Commercial Banking, Investment Bankin..."


In [ ]:
#Save to excel
scrapedAbouts.to_excel("linkedIn-scrape.xlsx")